In [2]:
import cv2
import mediapipe as mp
import json
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Download required NLTK resources
nltk.download('punkt')

# Load FAQ dataset from JSON
with open('faq_data.json', 'r') as file:
    faq_data = json.load(file)

questions = [faq["question"] for faq in faq_data["faqs"]]
answers = [faq["answer"] for faq in faq_data["faqs"]]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\balas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Preprocess text (convert to lowercase)
def preprocess(text):
    return text.lower()

preprocessed_questions = [preprocess(q) for q in questions]

In [5]:
# Convert text to numerical data using TF-IDF
vectorizer = TfidfVectorizer()
question_vectors = vectorizer.fit_transform(preprocessed_questions)

# Function to get chatbot response
def get_response(user_input):
    user_input = preprocess(user_input)
    user_vector = vectorizer.transform([user_input])
    similarities = cosine_similarity(user_vector, question_vectors)
    
    best_match_index = np.argmax(similarities)
    confidence = similarities[0][best_match_index]
    
    if confidence > 0.5:
        return answers[best_match_index]
    else:
        return "Sorry, I don't understand. Can you rephrase?"

In [7]:
# Start Chatbot Interaction
user_input = input("You: ")
response = get_response(user_input)
print("Chatbot:", response)

# Hand Gesture Recognition Setup
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Open webcam after chatbot interaction
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

print("Now, perform hand gestures...")

Chatbot: Hi, how can I help you?
Now, perform hand gestures...


In [8]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    frame = cv2.flip(frame, 1)  # Mirror image
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
            
            if index_tip.y < middle_tip.y:  # One finger up
                print("One Finger Detected! , We are open from 9 AM to 6 PM, Monday to Friday.")
            elif index_tip.y > middle_tip.y:  # Two fingers up
                print("Two Fingers Detected!, Go to settings, select 'Change Password', and follow the instructions.")
    
    cv2.imshow('Hand Gesture Chatbot', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

One Finger Detected! , We are open from 9 AM to 6 PM, Monday to Friday.
One Finger Detected! , We are open from 9 AM to 6 PM, Monday to Friday.
One Finger Detected! , We are open from 9 AM to 6 PM, Monday to Friday.
One Finger Detected! , We are open from 9 AM to 6 PM, Monday to Friday.
One Finger Detected! , We are open from 9 AM to 6 PM, Monday to Friday.
One Finger Detected! , We are open from 9 AM to 6 PM, Monday to Friday.
One Finger Detected! , We are open from 9 AM to 6 PM, Monday to Friday.
One Finger Detected! , We are open from 9 AM to 6 PM, Monday to Friday.
One Finger Detected! , We are open from 9 AM to 6 PM, Monday to Friday.
One Finger Detected! , We are open from 9 AM to 6 PM, Monday to Friday.
One Finger Detected! , We are open from 9 AM to 6 PM, Monday to Friday.
One Finger Detected! , We are open from 9 AM to 6 PM, Monday to Friday.
One Finger Detected! , We are open from 9 AM to 6 PM, Monday to Friday.
One Finger Detected! , We are open from 9 AM to 6 PM, Monday to 

KeyboardInterrupt: 

In [ ]:
cap.release()
cv2.destroyAllWindows()